In [1]:
import pandas as pd

## Filtering excel sheet

In [ ]:
df = pd.read_excel('./Gringotts.xlsx')
excel_file = pd.ExcelFile('./Gringotts.xlsx')

df['Numbers'] = df['College'].str.extract(r'(\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+)$')
df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Numbers'].str.split().apply(pd.Series)
df.drop(columns=['Numbers'], inplace=True)

In [29]:
import re
def extract_and_calculate_numbers(text):
  # numbers = re.findall(r'\d+\.\d+', text)
  numbers = re.findall(r'\d+(?:\.\d+)?', text)
  numbers = numbers[5:]
  if len(numbers) >= 5:
      tlr = float(numbers[0])
      rpc = float(numbers[1])
      go = float(numbers[2])
      oi = float(numbers[3])
      pr = float(numbers[4])
  else:
      tlr, rpc, go, oi, pr = None, None, None, None, None
  
  return pd.Series({'TLR': tlr, 'RPC': rpc, 'GO': go, 'OI': oi, 'PR': pr})


excel_file = pd.ExcelFile('./Gringotts.xlsx')
df = pd.read_excel(excel_file, sheet_name="2016")
df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Name'].apply(extract_and_calculate_numbers)
df

# df['Numbers'] = df['Name'].str.extract(r'(\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+\s\d+\.\d+)$')
# df[['TLR', 'RPC', 'GO', 'OI', 'PR']] = df['Numbers'].str.split().apply(pd.Series)
# df.drop(columns=['Numbers'], inplace=True)
df.to_excel('nirf/2016.xlsx', index=False)

In [2]:
df = pd.read_excel('./nirf/2021.xlsx')
df['Name'] = df['Name'].str.split('Close', expand=True)[0]
df.to_excel('nirf/2021.xlsx')

## Adding dummy data

In [27]:
subtopic_weights = {
    "TLR": {"SS": 20, "FSR": 30, "FQE": 20, "FRU": 30},
    "RPC": {"PU": 35, "QP": 40, "IPR": 15, "FPPP": 10},
    "GO": {"GPH": 40, "GUE": 15, "GMS": 25, "GPHD": 20},
    "OI": {"RD": 30, "WD": 30, "ESCS": 20, "PCS": 20},
}

In [31]:
df = pd.read_excel('./nirf/xl/2017.xlsx')
def add_dummy_data(df, property_name):
    subtopics = list(subtopic_weights[property_name].keys())
    total_value = df[property_name].values[0]

    for subtopic in subtopics:
        percentage = subtopic_weights[property_name][subtopic]
        dummy_value = (percentage / 100) * total_value
        df[subtopic] = dummy_value

for property in subtopic_weights.keys():
    add_dummy_data(df, property)

df.to_csv('nirf/2017.csv')

In [39]:
df = pd.read_csv('./nirf/2023.csv')
df.isna().any().any()

False

## Rank prediction

In [106]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
weightage = {
  'TLR': 0.30, 
  'RPC': 0.30, 
  'GO': 0.20, 
  'OI': 0.10, 
  'PR': 0.10
}

In [115]:
df = pd.read_csv('./nirf/2020.csv')

df['Weighted_Score'] = (
  df['TLR'] * weightage['TLR'] +
  df['RPC'] * weightage['RPC'] +
  df['GO'] * weightage['GO'] +
  df['OI'] * weightage['OI'] +
  df['PR'] * weightage['PR']
)

df.head()

,Institute ID,Name,City,State,Score,Rank,TLR,RPC,GO,OI,...,FPPP,GPH,GUE,GMS,GPHD,RD,WD,ESCS,PCS,Weighted_Score
0,IR-E-U-0456,Indian Institute of Technology Madras,Chennai,Tamil Nadu,89.93,1,95.42,94.64,83.90,61.31,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,89.929
1,IR-E-I-1074,Indian Institute of Technology Delhi,New Delhi,Delhi,88.08,2,90.79,96.15,80.36,64.81,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,88.081
2,IR-E-U-0306,Indian Institute of Technology Bombay,Mumbai,Maharashtra,85.08,3,91.00,93.37,77.60,49.99,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,85.081
3,IR-E-I-1075,Indian Institute of Technology Kanpur,Kanpur,Uttar Pradesh,82.18,4,86.22,82.08,88.44,54.21,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,82.177
4,IR-E-U-0573,Indian Institute of Technology Kharagpur,Kharagpur,West Bengal,80.56,5,77.32,87.11,83.21,56.62,...,9.464,33.56,12.585,20.975,16.78,18.393,18.393,12.262,12.262,80.564


In [116]:
X = df[['Weighted_Score']]
y = df['Rank']

In [117]:
model2 = RandomForestRegressor(n_estimators=100, random_state=0)
model1 = LinearRegression()
model1.fit(X, y)
model2.fit(X, y)

RandomForestRegressor(random_state=0)

In [118]:
df1 = pd.read_csv('./nirf/2023.csv')
scores = {'TLR': 10, 'RPC': 20, 'GO': 20, 'OI': 25, 'PR': 30}
scores = pd.DataFrame(scores, index=[0])
scores['Weighted_Score'] = (
  scores['TLR'] * weightage['TLR'] +
  scores['RPC'] * weightage['RPC'] +
  scores['GO'] * weightage['GO'] +
  scores['OI'] * weightage['OI'] +
  scores['PR'] * weightage['PR']
)
predictions = model1.predict(scores[['Weighted_Score']])
scores['Predicted_Rank1'] = predictions
# scores['Predicted_Rank1_Round'] = scores['Predicted_Rank1'].apply(round)

predictions = model2.predict(scores[['Weighted_Score']])
scores['Predicted_Rank2'] = predictions
# scores['Predicted_Rank2_Round'] = scores['Predicted_Rank2'].apply(round)
# df1 = df1.sort_values(by='Predicted_Rank')
# df1.reset_index(drop=True, inplace=True)
scores
# current_year_data = current_year_data.sort_values(by='Predicted_Rank')

,TLR,RPC,GO,OI,PR,Weighted_Score,Predicted_Rank1,Predicted_Rank2
0,10,20,20,25,30,18.5,199.587357,199.49
